# Делаем бота в телеграмме

In [ ]:
# устанавливаем одну из библиотек
!pip install pyTelegramBotAPI

# https://core.telegram.org/bots/samples#python
# есть и другие библиотеки

In [ ]:
import telebot
import time
import requests

## Случайный кот

https://thiscatdoesnotexist.com/

этот сайт генерирует случайное фото кота - пообновляйте страницу

In [ ]:
url = 'https://thiscatdoesnotexist.com/'
r = requests.get(url)  
with open('random_cat.png', 'wb') as f:
    f.write(r.content)

# можно котов скачивать и вот так
# ! curl https://thiscatdoesnotexist.com/ --output random_cat.png

In [ ]:
def download_random_cat():
    url = 'https://thiscatdoesnotexist.com/'
    r = requests.get(url)  
    with open('random_cat.png', 'wb') as f:
        f.write(r.content)

In [ ]:
with open('random_cat.png' ) as f:
    pic = f

## Простой бот (с комментариями)

In [ ]:
# пишем токен. лучше хранить в файле
TOKEN = "попросите его у @BotFather"
# инициализируем бота
bot = telebot.TeleBot(TOKEN)

# это будет приветствие в начале
greeting = '''
Hello!!!
You can restart this bot with /start
I have a quiz under /test
I can send you a random with /cat
I can you remind all comands with comand /help
'''.strip()

# задаем первую функцию, которая будет отвечать на команду /start
@bot.message_handler(commands=['start'])
def start_message(message):
    # инициализируем клавиатуру
    keyboard = telebot.types.ReplyKeyboardMarkup(True)
    # прописываем кнопки на клавиатуре
    keyboard.row('Hello =)', 'Bye =)')
    # отправляем сообщение
    bot.send_message(
        message.chat.id, # в какой чат отправлем
        greeting, # потом текст
        reply_markup=keyboard # потом клавиатура
        )
    # дальше будет функция, которая будет считаывать эти кнопки и реагировать на них
    # сначала должна идти обработка всех команд со слешем, а только потом обработка текста  

# по этой команде будем напоминать приветствие
@bot.message_handler(commands=['help'])
def help_message(message):
    bot.send_message(
        message.chat.id, # в какой чат отправлем
        greeting, # потом текст
        )
    
# скачиваем случайного кота и отправляем
@bot.message_handler(commands=['cat'])
def help_message(message):
    # скачивать фото случайного кота
    download_random_cat()
    with open('random_cat.png', 'rb') as f:
        picture = f
        # файл должен быть открыт в момент отправки
        # b - бинарный формат
        bot.send_photo(
            message.chat.id, # в какой чат отправлем
            picture, # тут фото
            )

# рассмотрим новый тип клавиутуры, который будет появлять при нажатии на команду /test
@bot.message_handler(commands=['test'])
def send_message_to_test(message):
    # инициализирую клавиатуру
    keyboard = telebot.types.InlineKeyboardMarkup()
    # создаем кнопки и сразу добавляем их в клавиатуру
    keyboard.add(telebot.types.InlineKeyboardButton(text='Three', callback_data='3'))
    keyboard.add(telebot.types.InlineKeyboardButton(text='Four', callback_data='4'))
    keyboard.add(telebot.types.InlineKeyboardButton(text='Five', callback_data='5'))
    bot.send_message(message.chat.id, text="How much is 2+2?", reply_markup=keyboard)

# эта функция будет обрабатывать нажатия
@bot.callback_query_handler(func=lambda call: True) #call in ['3','4','5']
def query_handler(call):
    # после клика на какую-либо кнопку я буду печатать Answer accepted!
    bot.answer_callback_query(callback_query_id=call.id, text='Answer accepted!')
    bot.send_message(call.message.chat.id, f'Are you sure that is {call.data}?')
    # добавим интриги в 3 секунди
    time.sleep(3)
    # обработаем то, на какую кнопку нажал пользователь – call.data
    if call.data == '4':
        answer = 'You answered correctly!'
    else:
        answer = 'You made a mistake'
    # отправим ответ в зависимостти от выбора варианта
    bot.send_message(call.message.chat.id, answer)
    # удалим клавиатуру
    bot.edit_message_reply_markup(call.message.chat.id, call.message.message_id)

# задаем вторую команду, которая будет обрабатывать любой текст 
# в зависимости от того, что нам присылают
# это указываем в content_types
@bot.message_handler(content_types=['text'])
def send_text(message):
    # достает текст из message
    text = message.text
    # в зависимости от содержимого в строке text как-то отвечаем
    if 'hello' in text.lower():
        bot.send_message(message.chat.id, 'Hello again!!!')
    elif 'bye' in text.lower():
        bot.send_message(message.chat.id, 'GOOD BYE =(((((')
    else:
        # сделаем калькулятор
        try:
            bot.send_message(message.chat.id, eval(text))
        except:
            bot.send_message(message.chat.id, '???')


# запуск бота
bot.polling()

## Упрощенная версия бота (без комментариев)

Код основан на туториале отсюда

https://pocketadmin.tech/en/telegram-bot-on-python/

In [ ]:
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['start'])
def start_message(message):
    keyboard = telebot.types.ReplyKeyboardMarkup(True)
    keyboard.row('Hello', 'Bye')
    bot.send_message(message.chat.id, 'Hello!', reply_markup=keyboard)

@bot.message_handler(commands=['test'])
def start_message(message):
    markup = telebot.types.InlineKeyboardMarkup()
    markup.add(telebot.types.InlineKeyboardButton(text='Three', callback_data=3))
    markup.add(telebot.types.InlineKeyboardButton(text='Four', callback_data=4))
    markup.add(telebot.types.InlineKeyboardButton(text='Five', callback_data=5))
    bot.send_message(message.chat.id, text="How much is 2 plus 2?", reply_markup=markup)

@bot.message_handler(content_types=['text'])
def send_text(message):
    if message.text.lower() == 'hello':
        bot.send_message(message.chat.id, 'Hello again!')
    elif message.text.lower() == 'bye':
        bot.send_message(message.chat.id, 'Bye!')
    else:
        try:
            bot.send_message(message.chat.id, eval(message.text.lower()))
        except:
            bot.send_message(message.chat.id, '???')

@bot.callback_query_handler(func=lambda call: True) #call in [3,4,5,'3','4','5']
def query_handler(call):
    bot.answer_callback_query(callback_query_id=call.id, text='Answer accepted!')
    bot.send_message(call.message.chat.id, f'Are you sure that is {call.data}?')
    time.sleep(3)
    
    if call.data == '4':
        answer = 'You answered correctly!'
    else:
        answer = 'You made a mistake'

    bot.send_message(call.message.chat.id, answer)
    bot.edit_message_reply_markup(call.message.chat.id, call.message.message_id)

bot.polling()

In [ ]:
eval('1+2')

In [ ]:
(1+2j)*(1-2j)

In [ ]:
type((1+2j)*(1-2j))

In [ ]:
2.78**(3.14j)

## База данных и SQL-запросы

In [ ]:
import telebot
from telebot import types
import sqlite3
import time

In [ ]:
# подключаем базу данных
conn = sqlite3.connect('planner_hse.db')

# курсор для работы с таблицами
cursor = conn.cursor()

try:
    # sql запрос для создания таблицы
    query = '''
    CREATE TABLE \"planner\" 
    (\"ID\" INTEGER UNIQUE, 
    \"user_id\" INTEGER, 
    \"plan\" TEXT, 
    PRIMARY KEY (\"ID\"))
    '''
    # исполняем его –> ура, теперь у нас есть таблица, куда будем все сохранять!
    cursor.execute(query)
except:
    pass




In [ ]:
user_id = 234567
text = '"qarwtsedrtfyghu"'

with sqlite3.connect('planner_hse.db') as con:
    cursor = con.cursor()
    cursor.execute(f'''
    INSERT INTO 
    planner (user_id, plan) 
    VALUES ({user_id}, {text})
    '''.strip())
    con.commit()


In [ ]:
with sqlite3.connect('planner_hse.db') as con:
    cursor = con.cursor()
    cursor.execute(f'''
        SELECT * 
        FROM planner 
    ''')
    tasks = cursor.fetchall()
    

In [ ]:
# тут будет список кортежей
tasks

In [ ]:
with sqlite3.connect('planner_hse.db') as con:
    cursor = con.cursor()
    cursor.execute(f'''
        SELECT plan 
        FROM planner 
        WHERE user_id=={123}
    ''')
    tasks = cursor.fetchall()
    

In [ ]:
tasks

In [ ]:
user_id = 2063180701
text = '"qarwtsedrtfyghu"'

with sqlite3.connect('planner_hse.db') as con:
    cursor = con.cursor()
    cursor.execute(f'''
        DELETE FROM planner 
        WHERE user_id=={user_id} AND plan=={text}
        '''.strip())
    con.commit()

## Бот из туториала с базой данных

In [ ]:
bot = telebot.TeleBot(TOKEN)

# напишем, что делать нашему боту при команде старт
@bot.message_handler(commands=['start'])
def send_keyboard(message, text="Привет, чем я могу тебе помочь?"):
    keyboard = types.ReplyKeyboardMarkup(row_width=2)  # наша клавиатура
    itembtn1 = types.KeyboardButton('Добавить дело в список') # создадим кнопку
    itembtn2 = types.KeyboardButton('Показать список дел')
    itembtn3 = types.KeyboardButton('Удалить дело из списка')
    itembtn4 = types.KeyboardButton("Удалить все дела из списка")
    itembtn5 = types.KeyboardButton('Другое')
    itembtn6 = types.KeyboardButton('Пока все!')
    keyboard.add(itembtn1, itembtn2) # добавим кнопки 1 и 2 на первый ряд
    keyboard.add(itembtn3, itembtn4, itembtn5, itembtn6) # добавим кнопки 3, 4, 5 на второй ряд
    # но если кнопок слишком много, они пойдут на след ряд автоматически

    # пришлем это все сообщением и запишем выбранный вариант
    msg = bot.send_message(message.from_user.id,
                     text=text, reply_markup=keyboard)

    # отправим этот вариант в функцию, которая его обработает
    bot.register_next_step_handler(msg, callback_worker)

@bot.message_handler(content_types=['text'])
def handle_docs_audio(message):
    send_keyboard(message, text="Я не понимаю :-( Выберите один из пунктов меню:")

# напишем функции для каждого случая
# эта добавляет строчку с планом в хранилище
def add_plan(msg):
    with sqlite3.connect('planner_hse.db') as con:
        cursor = con.cursor()
        cursor.execute('INSERT INTO planner (user_id, plan) VALUES (?, ?)',
                       (msg.from_user.id, msg.text))
        con.commit()
    bot.send_message(msg.chat.id, 'Запомню :-)')
    send_keyboard(msg, "Чем еще могу помочь?")

# просто функция, которая делает нам красивые строки для отправки пользователю
def get_plans_string(tasks):
    tasks_str = []
    for val in list(enumerate(tasks)):
        tasks_str.append(str(val[0] + 1) + ') ' + val[1][0] + '\n')
    return ''.join(tasks_str)

# отправляем пользователю его планы
def show_plans(msg):
    with sqlite3.connect('planner_hse.db') as con:
        cursor = con.cursor()
        cursor.execute('SELECT plan FROM planner WHERE user_id=={}'.format(msg.from_user.id))
        tasks = get_plans_string(cursor.fetchall())
        bot.send_message(msg.chat.id, tasks)
        send_keyboard(msg, "Чем еще могу помочь?")

# удаляет все планы для конкретного пользователя
def delete_all_plans(msg):
    with sqlite3.connect('planner_hse.db') as con:
        cursor = con.cursor()
        cursor.execute('DELETE FROM planner WHERE user_id=={}'.format(msg.from_user.id))
        con.commit()
    bot.send_message(msg.chat.id, 'Удалены все дела. Хорошего отдыха!')
    send_keyboard(msg, "Чем еще могу помочь?")

# выыделяет одно дело, которое пользователь хочет удалить
def delete_one_plan(msg):
    markup = types.ReplyKeyboardMarkup(row_width=2)
    with sqlite3.connect('planner_hse.db') as con:
        cursor = con.cursor()
        # достаем все задачи пользователя
        cursor.execute('SELECT plan FROM planner WHERE user_id=={}'.format(msg.from_user.id))
        # достанем результат запроса
        tasks = cursor.fetchall()
        for value in tasks:
            markup.add(types.KeyboardButton(value[0]))
        msg = bot.send_message(msg.from_user.id,
                               text = "Выбери одно дело из списка",
                               reply_markup=markup)
        bot.register_next_step_handler(msg, delete_one_plan_)

# удаляет это дело
def delete_one_plan_(msg):
    with sqlite3.connect('planner_hse.db') as con:
        cursor = con.cursor()
        cursor.execute('DELETE FROM planner WHERE user_id==? AND plan==?', (msg.from_user.id, msg.text))
        bot.send_message(msg.chat.id, 'Ура, минус одна задача!')
        send_keyboard(msg, "Чем еще могу помочь?")

# привязываем функции к кнопкам на клавиатуре
def callback_worker(call):
    if call.text == "Добавить дело в список":
        msg = bot.send_message(call.chat.id, 'Давайте добавим дело! Напишите его в чат')
        bot.register_next_step_handler(msg, add_plan)

    elif call.text == "Показать список дел":
        try:
            show_plans(call)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Можно отдыхать :-)')
            send_keyboard(call, "Чем еще могу помочь?")

    elif call.text == "Удалить дело из списка":
        try:
            delete_one_plan(call)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Можно отдыхать :-)')
            send_keyboard(call, "Чем еще могу помочь?")

    elif call.text == "Удалить все дела из списка":
        try:
            delete_all_plans(call)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Можно отдыхать :-)')
            send_keyboard(call, "Чем еще могу помочь?")

    elif call.text == "Другое":
        bot.send_message(call.chat.id, 'Больше я пока ничего не умею :-(')
        send_keyboard(call, "Чем еще могу помочь?")

    elif call.text == "Пока все!":
        bot.send_message(call.chat.id, 'Хорошего дня! Когда захотите продолжнить нажмите на команду /start',
                         reply_markup=None)



bot.polling(none_stop=True)